# Export all Recipients and create xlsx-Files

In [1]:
import pandas as pd
import mysql.connector

## Connect

In [2]:
db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="",
    database = "pharmagelder"
)

## Load Data

In [3]:
df_recipient = pd.read_sql("""SELECT
  rec_id as user_id,
  rec_name as name,
  rec_address as address,
  rec_location as location,
  rec_type as type,
  trc_name as category,
  tra_value as value,
  tra_year as year
 FROM transaction
  LEFT JOIN transaction_category ON tra_fk_transaction_category = transaction_category.trc_id
  LEFT JOIN recipient ON tra_fk_recipient = recipient.rec_id
  #GROUP BY tra_fk_recipient, tra_fk_transaction_category
  ORDER BY rec_name DESC""", con=db_connection)

## Export Sum

In [4]:
df_grouped = df_recipient.groupby(['user_id', 'year']).agg({
    'type': 'first',
    'name': 'first',
    'address': 'first',
    'location': 'first',
    'type': 'first',
    'value': 'sum'})

df_pivot = pd.pivot_table(df_grouped, values='value', columns='year', index=['user_id', 'name', 'address', 'location', 'type']).reset_index()
df_pivot['total'] = df_pivot[2015] + df_pivot[2016] + df_pivot[2017] + df_pivot[2018]
df_pivot = df_pivot.sort_values('total', ascending=False)

#Add Url
df_pivot['url'] = df_pivot['user_id'].apply(lambda x: 'https://www.stg.pharmagelder.ch/recipient/%s-x.html' % x)

df_pivot_hcp = df_pivot[df_pivot.type == 'hcp']
df_pivot_hco = df_pivot[df_pivot.type == 'hco']

## Export Categories

In [5]:
df_grouped_detail = df_recipient.groupby(['user_id', 'category']).agg({
    'type': 'first',
    'name': 'first',
    'address': 'first',
    'location': 'first',
    'type': 'first',
    'value': 'sum'}).reset_index()

df_pivot_detail = pd.pivot_table(df_grouped_detail, values='value', columns='category', index=['user_id', 'name', 'address', 'location', 'type']).reset_index()

df_pivot_detail_hcp = df_pivot_detail[df_pivot_detail.type == 'hcp']
df_pivot_detail_hco = df_pivot_detail[df_pivot_detail.type == 'hco']

## Write to Excel

In [6]:
#Write all
writer = pd.ExcelWriter('../../data/99. analyzes/excel/recipients/all_recipients.xlsx', options={'encoding':'utf-8'})
df_pivot_hcp.to_excel(writer, 'hcp', index=False)
df_pivot_hco.to_excel(writer, 'hco', index=False)
df_pivot_detail_hcp.to_excel(writer, 'hcp_kategorien', index=False)
df_pivot_detail_hco.to_excel(writer, 'hco_kategorien', index=False)
writer.save()

print("finito")

finito
